In [19]:
###PATH = "C:/Users/magaxels/AutoML"
PATH = "/Users/magnusax/AutoML/"
import sys; sys.path.insert(0, PATH)
from gazer import GazerMetaLearner
from gazer.ensembler import GazerMetaEnsembler
import numpy as np
import pandas as pd
import os
import time
from gazer.metrics import get_scorer
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import random


In [20]:
X, y = load_digits(return_X_y=True)

In [21]:
X.shape, y.shape

((1797, 64), (1797,))

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(898, 64) (898,) (899, 64) (899,)


In [23]:
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
##scaler = StandardScaler()
#scaler = StandardScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_val = scaler.transform(X_val)

In [24]:
learner = GazerMetaLearner(method='all', 
                           estimators=['adaboost', 'multinomial_nb', 'logreg', 'knn', 'neuralnet'], 
                           verbose=1)
learner.update('neuralnet', {'n_hidden': 3, 'epochs': 50, 'input_units': 1000, 'p': 0.1, 'batch_norm': True})

Available algorithms (use '.clf' attribute for access):
sgd_hinge, bernoulli_nb, random_forest, knn, adaboost, svm, neuralnet, multinomial_nb, tree, logreg, gaussian_nb


In [25]:
ensembler = GazerMetaEnsembler(learner, X_train.shape)

In [26]:
ensembler.fit(X_train, y_train, os.path.join(os.getcwd(),"tmp7"))

Training neural net..


neuralnet:   1%|▋                                                                       | 1/100 [00:00<00:14,  7.04it/s]

Train time: 0.17 min


random_forest: 100%|██████████████████████████████████████████████████████████████████| 100/100 [09:29<00:00,  5.70s/it]


In [28]:
scores, ensemble = ensembler.hillclimb(X_val, y_val, 
                                       n_best=5, 
                                       p=0.5, 
                                       iterations=20, 
                                       scoring='accuracy')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (140, 67))



KeyError: 'neuralnet'

In [ ]:
model, files = ensembler.orchestrator['neuralnet']

In [ ]:
files

In [ ]:
import keras
for wts, score in files:
    model.load_weights(wts)
    print(model.evaluate(X_val, keras.utils.to_categorical(y_val), verbose=0))

In [ ]:
pr = np.array([list(l).index(max(list(l))) 
               for l in model.predict(X_val)])
pr.shape


In [ ]:
pr

In [ ]:
valscores = {}; ensemble = {}
for n_best in range(1, 11):
    vs, ens = ensembler.hillclimb(X_val, y_val, n_best=n_best, p=0.8, iterations=7, scoring='accuracy')
    valscores[str(n_best)] = vs
    ensemble[str(n_best)] = ens

In [ ]:
valsscores = valscores['2']
x = [x for x,_ in valsscores]
y = [y for _,y in valsscores]
import matplotlib.pyplot as plt
% matplotlib inline
plt.figure(figsize=(12,6))
plt.plot(x,y, 'x--')
plt.show()

In [ ]:
len(ensemble['10'])